<a href="https://colab.research.google.com/github/Karisjk/loan-prediction-system-using-Logistic-Regression/blob/main/Machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install scikit

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [3]:
import pandas as pd

file_path="/content/data/loan_approval_dataset.csv"
data=pd.read_csv(file_path)
print(data.head())

   loan_id   no_of_dependents      education  self_employed   income_annum  \
0        1                  2       Graduate             No        9600000   
1        2                  0   Not Graduate            Yes        4100000   
2        3                  3       Graduate             No        9100000   
3        4                  3       Graduate             No        8200000   
4        5                  5   Not Graduate            Yes        9800000   

    loan_amount   loan_term   cibil_score   residential_assets_value  \
0      29900000          12           778                    2400000   
1      12200000           8           417                    2700000   
2      29700000          20           506                    7100000   
3      30700000           8           467                   18200000   
4      24200000          20           382                   12400000   

    commercial_assets_value   luxury_assets_value   bank_asset_value  \
0                  1760000

In [4]:
#check for missing values
print(data.isnull().sum())

loan_id                      0
 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64


In [5]:
#remove redundant columns
data.drop('loan_id', axis=1, inplace=True)

In [22]:
# Strip leading and trailing spaces in the target variable
data['loan_status'] = data['loan_status'].str.strip()

# Check unique values to ensure it's cleaned up
print(data['loan_status'].unique())


['Approved' 'Rejected']


In [7]:
print(data.columns)


Index([' no_of_dependents', ' education', ' self_employed', ' income_annum',
       ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')


In [8]:
#standardize column names
data.columns= data.columns.str.strip().str.lower()
print(data.columns)

Index(['no_of_dependents', 'education', 'self_employed', 'income_annum',
       'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value',
       'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value',
       'loan_status'],
      dtype='object')


In [9]:
data['education'].fillna(data['education'].mode()[0], inplace=True)

<ipython-input-9-03d86296fc38>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['education'].fillna(data['education'].mode()[0], inplace=True)


In [10]:
#encoding ctegorical variables
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()

data['education']= le.fit_transform(data['education'])
data['self_employed']= le.fit_transform(data['self_employed'])

In [11]:
#create derived features
#debt-to-income ratio
data['debt_to_income']=data['loan_amount']/data['income_annum']

In [13]:
#asset-to-loan ratio
data['asset_to_loan']= (data['residential_assets_value']+data['commercial_assets_value']+data['luxury_assets_value'])/data['loan_amount']

normalize numerical features

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

#normalize columns
num_cols= ['income_annum', 'loan_amount', 'loan_term', 'cibil_score',
           'residential_assets_value','commercial_assets_value',
           'luxury_assets_value', 'debt_to_income', 'asset_to_loan']

data[num_cols]=scaler.fit_transform(data[num_cols])

verify the dataset


In [16]:
print(data.head())
print(data.info())

   no_of_dependents  education  self_employed  income_annum  loan_amount  \
0                 2          0              0      0.969072     0.755102   
1                 0          1              1      0.402062     0.303571   
2                 3          0              0      0.917526     0.750000   
3                 3          0              0      0.824742     0.775510   
4                 5          1              1      0.989691     0.609694   

   loan_term  cibil_score  residential_assets_value  commercial_assets_value  \
0   0.555556     0.796667                  0.085616                 0.907216   
1   0.333333     0.195000                  0.095890                 0.113402   
2   1.000000     0.343333                  0.246575                 0.231959   
3   0.333333     0.278333                  0.626712                 0.170103   
4   1.000000     0.136667                  0.428082                 0.422680   

   luxury_assets_value  bank_asset_value loan_status  debt_to_

splitting the dataset

In [19]:
from sklearn.model_selection import train_test_split
# separate features(x) and(y)
X=data.drop('loan_status', axis=1)
y=data['loan_status']
#perform the split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)
print(f'training set size: {X_train.shape[0]} rows')
print(f'testing set size: {X_test.shape[0]} rows')

training set size: 3415 rows
testing set size: 854 rows


training with logistic regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
model= LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)
print("accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, pos_label=' Rejected'))
print("Recall:", recall_score(y_test, y_pred, pos_label=' Rejected'))
print("F1 Score:", f1_score(y_test, y_pred, pos_label=" Rejected"))
#confusion matrix
print("confusion Matrix:\n", confusion_matrix(y_test, y_pred))

accuracy: 0.7822014051522248
Precision: 0.7773109243697479
Recall: 0.5817610062893082
F1 Score: 0.6654676258992805
confusion Matrix:
 [[483  53]
 [133 185]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


hyperparameter tuning

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Initialize the Logistic Regression model
model = LogisticRegression(random_state=42)

# Define the parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],              # Regularization strength
    'solver': ['liblinear', 'lbfgs', 'saga'],          # Optimization algorithms
    'penalty': ['l2', 'elasticnet'],                   # Regularization types
    'max_iter': [100, 200, 300]                        # Number of iterations for convergence
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           scoring='accuracy', cv=5, n_jobs=-1, verbose=1)

# Fit the model to find the best parameters
grid_search.fit(X_train, y_train)

# Ensure the grid search completed and the best_estimator_ is available
if grid_search.best_estimator_:
    print("Best parameters found: ", grid_search.best_params_)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Evaluate the best model on the test set
    y_pred = best_model.predict(X_test)

    # Print evaluation metrics
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, pos_label=' Approved'))
    print("Recall:", recall_score(y_test, y_pred, pos_label=' Approved'))
    print("F1 Score:", f1_score(y_test, y_pred, pos_label=' Approved'))

    # Confusion Matrix
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

else:
    print("Grid search did not complete successfully.")



Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'C': 100, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 0.8934426229508197
Precision: 0.9097605893186004
Recall: 0.9216417910447762
F1 Score: 0.9156626506024096
Confusion Matrix:
 [[494  42]
 [ 49 269]]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
270 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1194, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/